# Exercises - Risk Metrics

# 1. Risk Metrics of a Stock

This problem uses data from `data/spx_returns_weekly.xlsx`.

Choose any stock to evaluate below.

## 1.1 Return Moments

Report the moments of the returns. Annualize the mean and volatility.
* mean
* volatility
* skewness
* (excess) kurtosis

Note that the pandas function for kurtosis already reports **excess** kurtosis.

## 1.2 Maximum Drawdown

Report the maximum drawdown for each return series.

If we resampled this data to weekly and recalculated the maximum drawdown, do you think it would be larger or smaller (in magnitude)?

## 1.3 Quantiles

Report the quantiles of the series. Use `.describe()` for a useful summary.

* Report the 5th quantile scaled by standard deviation.
* How much sampling error is in the mean return? Is this likely to cause much sampling error in the quantile estimation?

## 1.4 Comparison

Try repeating 1.1-1.3 for another asset. 

***

# 2. Time Aggregation

Use the data in `../data/risk_etf_data.xlsx`.

Use the price series to calculate **monthly** returns. (You may find `df.resample('M').last()` helpful.

## 2.1 Bitcoin and Measuring Covariation Risk

People often say that bitcoin has disappointed by being highly correlated to SPY.

Report the correlation between BTC and SPY for both daily and monthly returns.

What do you conclude?

## 2.2 Betas

For each series, calculate its beta to SPY.
* Estimate the regression with an intercept (alpha) but no need to report it.

How do these betas compare to the daily return betas seen in the note?

## 2.3 Time Scaling to Higher Moments

Report the skewness and kurtosis for the monthly returns.

How do these compare to the daily skewness and kurtosis measures?

What do you conclude?

***